# Spotify 2024 Data Analysis

This dataset was taken from Kaggle and describes the most streamed Spotify songs from 2024. The dataset and relevant documentation can be found at the following link: (

In [88]:
import pandas as pd
import numpy
import os
import datetime as dt

file_path = "C:/Users/AVILA/Desktop"

In [5]:
df = pd.read_csv(file_path + "/Most Streamed Spotify Songs 2024.csv", encoding_errors = "replace")

In [6]:
df

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,For the Last Time,For the Last Time,$uicideboy$,9/5/2017,QM8DG1703420,"4,585",19.4,"305,049,963","65,770","5,103,054",...,NaN,2.0,"14,217",NaN,"20,104,066","13,184","50,633,006","656,337",NaN,1
4596,Dil Meri Na Sune,"Dil Meri Na Sune (From ""Genius"")",Atif Aslam,7/27/2018,INT101800122,"4,575",19.4,"52,282,360","4,602","1,449,767",...,NaN,1.0,927,NaN,NaN,NaN,NaN,"193,590",NaN,0
4597,Grace (feat. 42 Dugg),My Turn,Lil Baby,2/28/2020,USUG12000043,"4,571",19.4,"189,972,685","72,066","6,704,802",...,NaN,1.0,74,6.0,"84,426,740","28,999",NaN,"1,135,998",NaN,1
4598,Nashe Si Chadh Gayi,November Top 10 Songs,Arijit Singh,11/8/2016,INY091600067,"4,591",19.4,"145,467,020","14,037","7,387,064",...,NaN,NaN,NaN,7.0,"6,817,840",NaN,NaN,"448,292",NaN,0


# Data Pre-Analysis + Cleaning

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Track                       4600 non-null   object 
 1   Album Name                  4600 non-null   object 
 2   Artist                      4595 non-null   object 
 3   Release Date                4600 non-null   object 
 4   ISRC                        4600 non-null   object 
 5   All Time Rank               4600 non-null   object 
 6   Track Score                 4600 non-null   float64
 7   Spotify Streams             4487 non-null   object 
 8   Spotify Playlist Count      4530 non-null   object 
 9   Spotify Playlist Reach      4528 non-null   object 
 10  Spotify Popularity          3796 non-null   float64
 11  YouTube Views               4292 non-null   object 
 12  YouTube Likes               4285 non-null   object 
 13  TikTok Posts                3427 

Identified that some of the notationa was not standard throughout the dataset, so standardized using the following lines of code.

In [10]:
df.loc[:, "All Time Rank" : "Explicit Track"] = df.iloc[:, 5:30].replace(",", "", regex = True)

In [11]:
df.loc[:, "All Time Rank" : "Explicit Track"].head()

,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,TikTok Posts,TikTok Likes,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,1,725.4,390470936,30716,196631588,92.0,84274754,1713126,5767700,651565900,...,684,62.0,17598718,114.0,18004655,22931,4818457,2669262,NaN,0
1,2,545.9,323703884,28113,174597137,92.0,116347040,3486739,674700,35223547,...,3,67.0,10422430,111.0,7780028,28444,6623075,1118279,NaN,1
2,3,538.4,601309283,54331,211607669,92.0,122599116,2228730,3025400,275154237,...,536,136.0,36321847,172.0,5022621,5639,7208651,5285340,NaN,0
3,4,444.9,2031280633,269802,136569078,85.0,1096100899,10629796,7189811,1078757968,...,2182,264.0,24684248,210.0,190260277,203384,NaN,11822942,NaN,0
4,5,423.3,107034922,7223,151469874,88.0,77373957,3670188,16400,NaN,...,1,82.0,17660624,105.0,4493884,7006,207179,457017,NaN,1


# Analysis Plan
After loading the dataset into JupyterLabs, I now want to identify an analysis plan. The primary questions that I want to analyze this dataset for are...
1. Are there any major discrepancies in terms of the number of plays of a song and when the song is released?
2. Are songs popular across all platforms evenly? Are there songs that have a high play rate on some platforms over other platforms?
3. On average, are explicit songs more popular than clean songs?


# 1. Determining major discrepancies in number of plays and song release date.

In [14]:
df.columns

Index(['Track', 'Album Name', 'Artist', 'Release Date', 'ISRC',
       'All Time Rank', 'Track Score', 'Spotify Streams',
       'Spotify Playlist Count', 'Spotify Playlist Reach',
       'Spotify Popularity', 'YouTube Views', 'YouTube Likes', 'TikTok Posts',
       'TikTok Likes', 'TikTok Views', 'YouTube Playlist Reach',
       'Apple Music Playlist Count', 'AirPlay Spins', 'SiriusXM Spins',
       'Deezer Playlist Count', 'Deezer Playlist Reach',
       'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations',
       'Soundcloud Streams', 'Shazam Counts', 'TIDAL Popularity',
       'Explicit Track'],
      dtype='object')

I identified the columns that would indicate the total number of listens that any individual song would have across all platforms.

In [16]:
columns = pd.DataFrame([df["Spotify Streams"],
                        df["YouTube Views"], 
                        df["TikTok Views"], 
                        df["AirPlay Spins"], 
                        df["SiriusXM Spins"], 
                        df["Pandora Streams"],
                        df["Soundcloud Streams"],
                        df["Shazam Counts"]])

After creating a seperate dataframe that contained those columns, I replaced the NaN values with 0.

In [18]:
columns = columns.fillna(0)

And since some of the variable types were stored as strings, I reformated all values to numeric values (which defaulted to floats)

In [20]:
for column in columns.columns:
    columns.iloc[:, column] = pd.to_numeric(columns.iloc[:, column])   

total_streams was created using a list comprehension to capture the sum of listens across each platform for each song, standardized to int64 values (as the numbers were too large for standard python), and then assigned back to the original dataframe.

In [22]:
total_streams = pd.Series([columns[column].sum() for column in columns.columns])
df["Total Streams"] = total_streams.astype("int64")
df["Total Streams"]

0        5832561659
1         663952112
2        4110620490
3       17934667726
4         189579145
           ...     
4595      669625504
4596      996416542
4597      513411965
4598     1309086812
4599     1262013624
Name: Total Streams, Length: 4600, dtype: int64

Now working with the date-time column "Release Date"

In [82]:
df["Release Date"] = pd.to_datetime(df["Release Date"])
df["Release Date"]

0      2024-04-26
1      2024-05-04
2      2024-03-19
3      2023-01-12
4      2024-05-31
          ...    
4595   2017-09-05
4596   2018-07-27
4597   2020-02-28
4598   2016-11-08
4599   2017-04-11
Name: Release Date, Length: 4600, dtype: datetime64[ns]

In [84]:
df["today"] = pd.to_datetime("today")

In [98]:
df["dif days"] = df["today"] - df["Release Date"]
df["dif days"]

0        94 days 14:23:19.085995
1        86 days 14:23:19.085995
2       132 days 14:23:19.085995
3       564 days 14:23:19.085995
4        59 days 14:23:19.085995
                  ...           
4595   2519 days 14:23:19.085995
4596   2194 days 14:23:19.085995
4597   1613 days 14:23:19.085995
4598   2820 days 14:23:19.085995
4599   2666 days 14:23:19.085995
Name: dif days, Length: 4600, dtype: timedelta64[ns]

In [64]:
df["listens per day"] = df["Total Streams"] / df["dif days"]

UFuncTypeError: ufunc 'divide' cannot use operands with types dtype('int64') and dtype('<m8[ns]')

In [23]:
first_quartile = df[df["Track Score"] < 23.3]
second_quartile = df[(df["Track Score"] > 23.3) & (df["Track Score"] < 29.9)]
third_quartile = df[(df["Track Score"] > 29.9) & (df["Track Score"] < 44.425)]
fourth_quartile = df[df["Track Score"] > 44.425]